# GitHub API EDA

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv('/Users/ammon/Projects/nmdp-bioinformatics/02-Repositories/gfe-db/.env.nmdpf');
from itertools import chain, starmap
import json
import requests
import pandas as pd

# Pandas display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

/var/folders/8s/9sb2nsn913q7b4zz75fd_qf00000gn/T/ipykernel_12407/494911564.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
os.getcwd()

'/Users/ammon/Projects/nmdp-bioinformatics/02-Repositories/gfe-db/notebooks'

In [3]:
# Environment variables
AWS_REGION = os.environ["AWS_REGION"] 
GITHUB_PERSONAL_ACCESS_TOKEN = os.environ["GITHUB_PERSONAL_ACCESS_TOKEN"]
GITHUB_REPOSITORY_OWNER = "ANHIG" # os.environ["GITHUB_REPOSITORY_OWNER"]
GITHUB_REPOSITORY_NAME = "IMGTHLA" # os.environ["GITHUB_REPOSITORY_NAME"]


In [5]:
def flatten_json(dictionary, sep='.', skip_fields=[]):
    """Flatten a nested json file. For a list of dictionaries, use this
    inside a for loop before converting to pandas DataFrame."""

    def unpack(parent_key, parent_value):
        """Unpack one level of nesting in json file"""
        # Unpack one level only!!!
        
        if isinstance(parent_value, dict):
            for key, value in parent_value.items():
                temp1 = parent_key + sep + key
                yield temp1, value
        elif isinstance(parent_value, list):
            i = 0 
            for value in parent_value:
                temp2 = parent_key + sep +str(i) 
                i += 1
                yield temp2, value
        else:
            yield parent_key, parent_value    


    # Keep iterating until the termination condition is satisfied
    while True:
        # Keep unpacking the json file until all values are atomic elements (not dictionary or list)
        dictionary = dict(chain.from_iterable(starmap(unpack, dictionary.items())))
        # Terminate condition: not any value in the json file is dictionary or list
        if not any(isinstance(value, dict) for value in dictionary.values()) and \
           not any(isinstance(value, list) for value in dictionary.values()):
            break

    return dictionary




In [6]:
def get_commits(owner, repo, per_page=100):
    """Return a list of GitHub commits for the specified repository"""

    base_url = 'https://api.github.com'

    # Endpoint
    endpoint = f'/repos/{owner}/{repo}/commits?per_page={per_page}'

    url = base_url + endpoint

    # Headers
    headers = {
        'Authorization': f'token {GITHUB_PERSONAL_ACCESS_TOKEN}',
        'Content-Type': 'application/json',
        'Accept': 'application/vnd.github.v3+json',
        'X-GitHub-Api-Version': '2022-11-28'
    }

    response = requests.get(url, headers=headers)

    return response.json()

In [7]:
def get_commit(owner, repo, commit_sha):
    """Return the commit for the specified repository and commit SHA"""

    base_url = 'https://api.github.com'

    # Endpoint
    endpoint = f'/repos/{owner}/{repo}/commits/{commit_sha}'
    url = base_url + endpoint

    # Headers
    headers = {
        'Authorization': f'token {GITHUB_PERSONAL_ACCESS_TOKEN}',
        'Content-Type': 'application/json',
        'Accept': 'application/vnd.github.v3+json',
        'X-GitHub-Api-Version': '2022-11-28'
    }

    response = requests.get(url, headers=headers)

    return response.json()

In [8]:
def get_branches(owner, repo):
    """Fetch branches for a GitHub repository"""

    base_url = 'https://api.github.com'

    # Endpoint
    endpoint = f'/repos/{owner}/{repo}/branches'
    url = base_url + endpoint

    # Headers
    headers = {
        'Authorization': f'token {GITHUB_PERSONAL_ACCESS_TOKEN}',
        'Content-Type': 'application/json',
        'Accept': 'application/vnd.github.v3+json',
        'X-GitHub-Api-Version': '2022-11-28'
    }

    response = requests.get(url, headers=headers)
    branches = response.json()

    return branches

In [9]:
def get_branch(owner, repo, branch_name):
    """Fetch branches for a GitHub repository"""

    base_url = 'https://api.github.com'

    # Endpoint
    endpoint = f'/repos/{owner}/{repo}/branches/{branch_name}'
    url = base_url + endpoint

    # Headers
    headers = {
        'Authorization': f'token {GITHUB_PERSONAL_ACCESS_TOKEN}',
        'Content-Type': 'application/json',
        'Accept': 'application/vnd.github.v3+json',
        'X-GitHub-Api-Version': '2022-11-28'
    }

    response = requests.get(url, headers=headers)
    branches = response.json()

    return branches

In [10]:
# Function to fetch pull requests
def fetch_pull_requests(owner, repo):
    url = f"https://api.github.com/repos/{owner}/{repo}/pulls?state=all"
    
    # Headers
    headers = {
        'Authorization': f'token {GITHUB_PERSONAL_ACCESS_TOKEN}',
        'Content-Type': 'application/json',
        'Accept': 'application/vnd.github.v3+json',
        'X-GitHub-Api-Version': '2022-11-28'
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return []

## Branches

In [11]:
from pygethub import list_branches, GitHubPaginator

In [12]:
paginator = GitHubPaginator(GITHUB_PERSONAL_ACCESS_TOKEN)

In [20]:
# Get all branches
paginator = GitHubPaginator(GITHUB_PERSONAL_ACCESS_TOKEN)
pages = paginator.get_paginator(list_branches, owner=GITHUB_REPOSITORY_OWNER, repo=GITHUB_REPOSITORY_NAME)
all_branches = list(pages)

NameError: name 'pygethub' is not defined

In [21]:
len(all_branches)

0

In [11]:
branches_df = pd.DataFrame([flatten_json(branch) for branch in all_branches])

In [12]:
branches_df.head()

""


In [16]:
releases = list(branches_df['name'].unique())[:-1]

In [17]:
print(",".join(releases))

300,310,320,330,340,350,360,370,380,390,3100,3110,3120,3130,3140,3150,3160,3170,3180,3190,3200,3210,3220,3230,3240,3250,3260,3270,3280,3290,3300,3310,3320,3330,3340,3350,3360,3370,3380,3390,3400,3410,3420,3430,3440,3450,3460,3470,3480,3490,3500,3510,3520,3530


## Pull Requests

In [51]:
# # Fetch pull requests and load them into a DataFrame
# pull_requests = fetch_pull_requests(GITHUB_REPOSITORY_OWNER, GITHUB_REPOSITORY_NAME, GITHUB_PERSONAL_ACCESS_TOKEN)

In [52]:
# # pull_requests_df = pd.DataFrame(pull_requests)
# pull_requests_df = pd.DataFrame([flatten_json(pull_request) for pull_request in pull_requests])

In [53]:
# pull_requests_df.head()

In [54]:
# pull_requests_df["title"].unique()

## Commits

In [55]:
commits = get_commits(GITHUB_REPOSITORY_OWNER, GITHUB_REPOSITORY_NAME)

In [56]:
len(commits)

100

In [57]:
commits[0]['sha']

'38398a75e9762ff070d8e9bd714d074332646cd7'

In [58]:
get_commit(GITHUB_REPOSITORY_OWNER, GITHUB_REPOSITORY_NAME, "38398a75e9762ff070d8e9bd714d074332646cd7")

{'sha': '38398a75e9762ff070d8e9bd714d074332646cd7',
 'node_id': 'C_kwDOAXZz6NoAKDM4Mzk4YTc1ZTk3NjJmZjA3MGQ4ZTliZDcxNGQwNzQzMzI2NDZjZDc',
 'commit': {'author': {'name': 'xeniageorgiouAN',
   'email': 'xenia.georgiou@anthonynolan.org',
   'date': '2023-04-17T16:03:52Z'},
  'committer': {'name': 'GitHub',
   'email': 'noreply@github.com',
   'date': '2023-04-17T16:03:52Z'},
  'message': 'Merge pull request #334 from ANHIG/3520\n\nIPD-IMGT/HLA Release 3.52.0',
  'tree': {'sha': 'e9ffca9666e355b1285a0c6a42951f6a28ea7f90',
   'url': 'https://api.github.com/repos/ANHIG/IMGTHLA/git/trees/e9ffca9666e355b1285a0c6a42951f6a28ea7f90'},
  'url': 'https://api.github.com/repos/ANHIG/IMGTHLA/git/commits/38398a75e9762ff070d8e9bd714d074332646cd7',
  'comment_count': 0,
  'verification': {'verified': True,
   'reason': 'valid',
   'signature': '-----BEGIN PGP SIGNATURE-----\n\nwsBcBAABCAAQBQJkPW3oCRBK7hj4Ov3rIwAAkxkIAI1/KmiZW1yRfwe8e++ROHQo\ncPbR+1PipOfVXcgaJVyK5vfiR1d+1DsLph0k+VurKcVIn5As5Pc+XSi7ImeeNFzV

In [128]:
# load dataframe from json and drop columns that are all null
commits_df = (
    pd.DataFrame([flatten_json(commit) for commit in commits])
    # pd.DataFrame(commits)
    .dropna(axis=1, how='all')
    # .sort_values(by='commit.committer.date', ascending=False)
)

In [129]:
commits_df.columns

Index(['sha', 'node_id', 'commit.author.name', 'commit.author.email',
       'commit.author.date', 'commit.committer.name', 'commit.committer.email',
       'commit.committer.date', 'commit.message', 'commit.tree.sha',
       'commit.tree.url', 'commit.url', 'commit.comment_count',
       'commit.verification.verified', 'commit.verification.reason',
       'commit.verification.signature', 'commit.verification.payload', 'url',
       'html_url', 'comments_url', 'author.login', 'author.id',
       'author.node_id', 'author.avatar_url', 'author.gravatar_id',
       'author.url', 'author.html_url', 'author.followers_url',
       'author.following_url', 'author.gists_url', 'author.starred_url',
       'author.subscriptions_url', 'author.organizations_url',
       'author.repos_url', 'author.events_url', 'author.received_events_url',
       'author.type', 'author.site_admin', 'committer.login', 'committer.id',
       'committer.node_id', 'committer.avatar_url', 'committer.gravatar_id',
     

In [130]:
cols = [
    "sha",
    "commit.author.date",
    "commit.message"
]
commits_df = commits_df[cols]

# convert date column to datetime
commits_df["date"] = pd.to_datetime(commits_df["commit.author.date"])
commits_df.drop(columns=["commit.author.date"], inplace=True)

In [131]:
# # Commit SHAs do not correspond to branch SHAs, so this approach doesn't work
# # join the branches df to the commits df on branches['commit.sha']
# commits_df = commits_df.merge(branches_df, left_on='sha', right_on='commit.sha', how='outer')

In [132]:
# commits_df.iloc[:4, :]

In [133]:
commit_refs = []
limit = 20
for idx, sha in enumerate(commits_df.iloc[:20, :]["sha"].to_list()):
    # fetch commit
    print(sha)
    commit = get_commit(GITHUB_REPOSITORY_OWNER, GITHUB_REPOSITORY_NAME, sha)
    commit_refs.append(commit)

    if idx+1 == limit:
        break

38398a75e9762ff070d8e9bd714d074332646cd7
def376dc6955b339b17f0a4b840e80eb6b9c744b
5f2c562056f8ffa89aeea0631f2a52300ee0de17
4b8432c7d56121c84d6ef1d75a1c7185c628c13d
4486f5c623705c6a14d9eeaba7d155cff30cdb43
50b790037030d958b662085c3f4cf34ba72a32ec
36220a1c5c2d6954f4873a552544cc0e55b61d0a
e941759874365cb152a3562c22d10847d10db326
2d38d3313229fdc5f8aa00052a2db21b35be3d2d
1ce31fc9e2805034578eff60a269c02176f03252
2c631a4b61d529ff1c0635750888f6f6d79c2703
12b4b94bb49b4640fa48a1182cc9b1e1fbf7e816
8f80f24d49797595d8a18b8d4d1f59846fbf3fe1
a9680a9e80e2c119d4aa41f90e5f40cef6e7df02
1fd937e6c0ff8452f64152aec9632f0586f981d1
50e92c677f4cd547c32bb9305269a762a9595074
8732e7d4739a911e01f69d2c5bda82ca15ca68a9
536e8833f3be4bbcffbfba43cd0b3043c5bf4068
c90cc62006b35061c8db4b6f8da1b86de7650b23
1a3be9a5d01a414854ff3bfacd5257c14adeefa2


In [137]:
# commit_refs_df = pd.DataFrame([flatten_json(commit) for commit in commit_refs])
commit_refs_df = pd.DataFrame(commit_refs)[["sha", "files", "parents"]]

In [141]:
commit_refs_df.loc[:20, :]['parents']

0     [{'sha': '5f2c562056f8ffa89aeea0631f2a52300ee0de17', 'url': 'https://api.github.com/repos/ANHIG/IMGTHLA/commits/5f2c562056f8ffa89aeea0631f2a52300ee0de17', 'html_url': 'https://github.com/ANHIG/IMGTHLA/commit/5f2c562056f8ffa89aeea0631f2a52300ee0de17'}, {'sha': 'def376dc6955b339b17f0a4b840e80eb6b9c744b', 'url': 'https://api.github.com/repos/ANHIG/IMGTHLA/commits/def376dc6955b339b17f0a4b840e80eb6b9c744b', 'html_url': 'https://github.com/ANHIG/IMGTHLA/commit/def376dc6955b339b17f0a4b840e80eb6b9c744b'}]
1                                                                                                                                                                                                                                                                [{'sha': '5f2c562056f8ffa89aeea0631f2a52300ee0de17', 'url': 'https://api.github.com/repos/ANHIG/IMGTHLA/commits/5f2c562056f8ffa89aeea0631f2a52300ee0de17', 'html_url': 'https://github.com/ANHIG/IMGTHLA/commit/5f2c562056f8ffa89aeea0631f

In [97]:
commit_refs_df['parents']

0     [{'sha': '5f2c562056f8ffa89aeea0631f2a52300ee0de17', 'url': 'https://api.github.com/repos/ANHIG/IMGTHLA/commits/5f2c562056f8ffa89aeea0631f2a52300ee0de17', 'html_url': 'https://github.com/ANHIG/IMGTHLA/commit/5f2c562056f8ffa89aeea0631f2a52300ee0de17'}, {'sha': 'def376dc6955b339b17f0a4b840e80eb6b9c744b', 'url': 'https://api.github.com/repos/ANHIG/IMGTHLA/commits/def376dc6955b339b17f0a4b840e80eb6b9c744b', 'html_url': 'https://github.com/ANHIG/IMGTHLA/commit/def376dc6955b339b17f0a4b840e80eb6b9c744b'}]
1                                                                                                                                                                                                                                                                [{'sha': '5f2c562056f8ffa89aeea0631f2a52300ee0de17', 'url': 'https://api.github.com/repos/ANHIG/IMGTHLA/commits/5f2c562056f8ffa89aeea0631f2a52300ee0de17', 'html_url': 'https://github.com/ANHIG/IMGTHLA/commit/5f2c562056f8ffa89aeea0631f

In [142]:
commits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   sha             100 non-null    object             
 1   commit.message  100 non-null    object             
 2   date            100 non-null    datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 2.5+ KB


In [143]:
commit_refs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   sha      20 non-null     object
 1   files    20 non-null     object
 2   parents  20 non-null     object
dtypes: object(3)
memory usage: 608.0+ bytes


In [144]:
# join commit dfs on sha
commits_joined_df = commits_df.merge(commit_refs_df, on='sha', how='inner')

In [145]:
commits_joined_df.columns

Index(['sha', 'commit.message', 'date', 'files', 'parents'], dtype='object')

In [146]:
commits_joined_df.loc[:10, ['sha', 'commit.message', 'date', 'parents', 'files']].to_csv("commits.csv", index=False, header=True)

In [152]:
commits_joined_df['parents'].iloc[5]

[{'sha': 'e941759874365cb152a3562c22d10847d10db326',
  'url': 'https://api.github.com/repos/ANHIG/IMGTHLA/commits/e941759874365cb152a3562c22d10847d10db326',
  'html_url': 'https://github.com/ANHIG/IMGTHLA/commit/e941759874365cb152a3562c22d10847d10db326'},
 {'sha': '36220a1c5c2d6954f4873a552544cc0e55b61d0a',
  'url': 'https://api.github.com/repos/ANHIG/IMGTHLA/commits/36220a1c5c2d6954f4873a552544cc0e55b61d0a',
  'html_url': 'https://github.com/ANHIG/IMGTHLA/commit/36220a1c5c2d6954f4873a552544cc0e55b61d0a'}]

In [68]:
# # # group commits by day
# # commit_refs_df = pd.DataFrame(commit_refs)
# # commit_refs_df["date"] = pd.to_datetime(commit_refs_df["commit"].apply(lambda x: x["author"]["date"]))

# # # extract message field
# # commit_refs_df["message"] = commit_refs_df["commit"].apply(lambda x: x["message"])

# # convert commit.author.date to datetime and sort by date descending
# commit_refs_df["date"] = pd.to_datetime(commit_refs_df["commit.author.date"])

# # extract filename from files field
# commit_refs_df["files"] = commit_refs_df["files"].apply(lambda x: [file["filename"] for file in x])

# # select columns
# cols = [
#     "sha",
#     "date",
#     "files",
# ]

# commit_refs_df = commit_refs_df[cols].explode("files").dropna().sort_values("date", ascending=False)

In [69]:
# commit_refs_df.groupby(pd.Grouper(key="date", freq="D"))[["sha", "files"]].count()

In [70]:
# flatten_json(commit_refs_df.loc[0, "commit"])

## GitHub Graph Analysis

In [4]:
import networkx as nx

In [153]:
commits_joined_df.iloc[:7, :].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   sha             7 non-null      object             
 1   commit.message  7 non-null      object             
 2   date            7 non-null      datetime64[ns, UTC]
 3   files           7 non-null      object             
 4   parents         7 non-null      object             
dtypes: datetime64[ns, UTC](1), object(4)
memory usage: 408.0+ bytes


In [127]:
# commits_joined_df.loc[:7, ['sha', 'commit', 'parents', 'files']].to_csv("commits_joined.csv", index=False, header=True)

In [188]:
commits_exp_df = commits_joined_df.explode("files").explode("parents")
commits_exp_df.columns = ['sha', 'commit.message', 'date', 'file', 'parent']

In [189]:
# unnest the fields in the file column as their own columns with prefix
commits_exp_df = pd.json_normalize(commits_exp_df['file'], sep='.')


In [190]:
commits_exp_df.iloc[5:, :].head()

,sha,filename,status,additions,deletions,changes,blob_url,raw_url,contents_url,previous_filename,patch
5,df1a5591eff92be3a94c065884d8264d308513c6,ihiw/hml/3.51.0_ReferenceSequences.fasta,renamed,0.0,0.0,0.0,https://github.com/ANHIG/IMGTHLA/blob/4b8432c7d56121c84d6ef1d75a1c7185c628c13d/ihiw%2Fhml%2F3.51.0_ReferenceSequences.fasta,https://github.com/ANHIG/IMGTHLA/raw/4b8432c7d56121c84d6ef1d75a1c7185c628c13d/ihiw%2Fhml%2F3.51.0_ReferenceSequences.fasta,https://api.github.com/repos/ANHIG/IMGTHLA/contents/ihiw%2Fhml%2F3.51.0_ReferenceSequences.fasta?ref=4b8432c7d56121c84d6ef1d75a1c7185c628c13d,ihiw/hml/3.50.0_ReferenceSequences.fasta,NaN
6,e23440821dc326f42b5444dbb1eac323650fae1e,ihiw/hml/3.51.0_Reference_Alleles.txt,renamed,3.0,3.0,6.0,https://github.com/ANHIG/IMGTHLA/blob/4b8432c7d56121c84d6ef1d75a1c7185c628c13d/ihiw%2Fhml%2F3.51.0_Reference_Alleles.txt,https://github.com/ANHIG/IMGTHLA/raw/4b8432c7d56121c84d6ef1d75a1c7185c628c13d/ihiw%2Fhml%2F3.51.0_Reference_Alleles.txt,https://api.github.com/repos/ANHIG/IMGTHLA/contents/ihiw%2Fhml%2F3.51.0_Reference_Alleles.txt?ref=4b8432c7d56121c84d6ef1d75a1c7185c628c13d,ihiw/hml/3.50.0_Reference_Alleles.txt,"@@ -1,8 +1,8 @@\n-# filename: 3.50.0_Reference_Alleles.txt\n-# date: 2022-10-12\n+# filename: 3.51.0_Reference_Alleles.txt\n+# date: 2023-01-12\n # version: 1.0\n # author: Ben Matern <B.M.Matern@umcutrecht.nl>\n-IPD-IMGT/HLA Database 3.50.0 Accession Number\tLocus\tIPD-IMGT/HLA Database 3.50.0 Allele Name\tDescription\n+IPD-IMGT/HLA Database 3.51.0 Accession Number\tLocus\tIPD-IMGT/HLA Database 3.51.0 Allele Name\tDescription\n HLA00001\tHLA-A\tHLA-A*01:01:01:01\tHLA-A Locus Reference;A*01 Reference\n HLA00005\tHLA-A\tHLA-A*02:01:01:01\tA*02 Reference\n HLA00037\tHLA-A\tHLA-A*03:01:01:01\tA*03 Reference"
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,09190c395f26163a25fa85d0666b35f9f8846f49,Allelelist_history.txt,modified,2.0,2.0,4.0,https://github.com/ANHIG/IMGTHLA/blob/50b790037030d958b662085c3f4cf34ba72a32ec/Allelelist_history.txt,https://github.com/ANHIG/IMGTHLA/raw/50b790037030d958b662085c3f4cf34ba72a32ec/Allelelist_history.txt,https://api.github.com/repos/ANHIG/IMGTHLA/contents/Allelelist_history.txt?ref=50b790037030d958b662085c3f4cf34ba72a32ec,NaN,"@@ -1,6 +1,6 @@\n # file: Allelelist_history.txt\n-# date: \n-# version: IPD-IMGT/HLA \n+# date: 2022-10-12\n+# version: IPD-IMGT/HLA 3.50.0\n # origin: https://github.com/ANHIG/IMGTHLA/Allelelist_history.txt\n # repository: https://raw.githubusercontent.com/ANHIG/IMGTHLA/Latest/Allelelist_history.txt\n # author: WHO, Steven G. E. Marsh (steven.marsh@ucl.ac.uk)"
9,09190c395f26163a25fa85d0666b35f9f8846f49,Allelelist_history.txt,modified,2.0,2.0,4.0,https://github.com/ANHIG/IMGTHLA/blob/50b790037030d958b662085c3f4cf34ba72a32ec/Allelelist_history.txt,https://github.com/ANHIG/IMGTHLA/raw/50b790037030d958b662085c3f4cf34ba72a32ec/Allelelist_history.txt,https://api.github.com/repos/ANHIG/IMGTHLA/contents/Allelelist_history.txt?ref=50b790037030d958b662085c3f4cf34ba72a32ec,NaN,"@@ -1,6 +1,6 @@\n # file: Allelelist_history.txt\n-# date: \n-# version: IPD-IMGT/HLA \n+# date: 2022-10-12\n+# version: IPD-IMGT/HLA 3.50.0\n # origin: https://github.com/ANHIG/IMGTHLA/Allelelist_history.txt\n # repository: https://raw.githubusercontent.com/ANHIG/IMGTHLA/Latest/Allelelist_history.txt\n # author: WHO, Steven G. E. Marsh (steven.marsh@ucl.ac.uk)"


In [184]:
list(commits_exp_df.columns)

['sha',
 'commit.message',
 'date',
 'file',
 'parent',
 0,
 'sha',
 'filename',
 'status',
 'additions',
 'deletions',
 'changes',
 'blob_url',
 'raw_url',
 'contents_url',
 'previous_filename',
 'patch']

In [ ]:
G = nx.from_pandas_edgelist(commits_exp_df, source='sha', target='file', edge_attr=[])

In [158]:
vars(G)

{'graph': {},
 '_node': {'38398a75e9762ff070d8e9bd714d074332646cd7': {'label': 'Merge pull request #334 from ANHIG/3520\n\nIPD-IMGT/HLA Release 3.52.0'},
  'def376dc6955b339b17f0a4b840e80eb6b9c744b': {'label': 'IPD-IMGT/HLA Release 3.52.0'},
  '5f2c562056f8ffa89aeea0631f2a52300ee0de17': {'label': 'Merge pull request #325 from ANHIG/3510\n\n3510'},
  '4b8432c7d56121c84d6ef1d75a1c7185c628c13d': {'label': 'IPD-IMGT/HLA Database Release 3.51.0\n\nUpdated the ihiw folder for 3.51.0'},
  '4486f5c623705c6a14d9eeaba7d155cff30cdb43': {'label': 'IPD-IMGT/HLA Release 3.51.0\n\nUpdates for 3.51.0'},
  '50b790037030d958b662085c3f4cf34ba72a32ec': {'label': 'Merge pull request #323 from ANHIG/3500\n\nCorrected missing date and version field in Allelelist_history.txt'},
  '36220a1c5c2d6954f4873a552544cc0e55b61d0a': {'label': 'Corrected missing date and version field in Allelelist_history.txt'},
  'e941759874365cb152a3562c22d10847d10db326': {'label': 'Merge pull request #318 from ANHIG/3500\n\nhla.dat 